In [1]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [2]:
!pip install transformers datasets seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a642d8894a8fd85a706a4cdaeba9fc39e3c6078727134ea85e9b266c25a00bf4
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver

# **Load the Dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import json

# Define the path where your files are located in Google Drive
train_file_path = '/content/drive/My Drive/Colab Notebooks/train.json'
test_file_path = '/content/drive/My Drive/Colab Notebooks/test.json'

In [6]:
# Load train and test data from JSON
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

train_data = load_data(train_file_path)
test_data = load_data(test_file_path)

# **Preprocess the Data**

In [7]:
def process_data(data):
  all_texts=[]
  all_labels=[]

  for doc in data:
    tokens=doc['tokens']
    labels = doc.get('labels',['O']*len(tokens)) #Use 'O' for missing labels
    all_texts.append(tokens)
    all_labels.append(labels)

  return all_texts,all_labels

In [8]:
train_texts, train_labels = process_data(train_data)
test_texts, test_labels = process_data(test_data)

In [9]:
from datasets import Dataset


# Convert to huggingface dataset format
train_dataset = Dataset.from_dict({'tokens': train_texts, 'ner_tags': train_labels})
test_dataset = Dataset.from_dict({'tokens': test_texts})



In [10]:
# Define the BIO label mapping
labels_list = ['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-EMAIL', 'I-EMAIL',
               'B-USERNAME', 'I-USERNAME', 'B-ID_NUM', 'I-ID_NUM', 'B-PHONE_NUM',
               'I-PHONE_NUM', 'B-URL_PERSONAL', 'I-URL_PERSONAL', 'B-STREET_ADDRESS',
               'I-STREET_ADDRESS']
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for i, label in enumerate(labels_list)}


In [11]:
def align_labels_with_tokens(labels, word_ids):
    aligned_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            aligned_labels.append(-100)
        elif word_id != previous_word_id:
            aligned_labels.append(labels[word_id])
        else:
            aligned_labels.append(-100)
        previous_word_id = word_id
    return aligned_labels

# **Load and Fine Tune the DeBERTa Model**

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification


In [13]:
# Load DeBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForTokenClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=len(labels_list))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print(test_dataset.column_names)


['tokens']


In [15]:
# Tokenize and align the labels
def tokenize_and_align_labels(examples, labels_available=True, max_length=256):
    # Tokenize the inputs with truncation, padding, and a max_length
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',  # Ensures padding to max_length
        max_length=max_length  # Set max_length to prevent overly long sequences
    )

    if labels_available:
        all_labels = examples['ner_tags']
        new_labels = []

        # Iterate over each set of labels
        for i, labels in enumerate(all_labels):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            aligned_labels = align_labels_with_tokens(labels, word_ids)

            # Print debug information to help troubleshoot length mismatches
            if len(aligned_labels) != len(word_ids):
                print(f"Length mismatch at index {i}: Tokens length = {len(word_ids)}, Labels length = {len(aligned_labels)}")

            new_labels.append(aligned_labels)

        # Map labels to their respective IDs (make sure to include all label types)
        label_to_id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4, -100: -100}  # Extend as needed
        new_labels = [[label_to_id.get(label, -100) for label in label_list] for label_list in new_labels]

        tokenized_inputs["labels"] = new_labels

    return tokenized_inputs


In [16]:
# Apply the tokenization and label alignment for the training set (with labels)
train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, labels_available=True), batched=True)

# Apply the tokenization for the test set (without labels)
test_dataset = test_dataset.map(lambda x: tokenize_and_align_labels(x, labels_available=False), batched=True)


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [17]:
# Data collator to ensure correct padding for token classification tasks
data_collator = DataCollatorForTokenClassification(tokenizer)


In [18]:
# Define the evaluation metric
from seqeval.metrics import accuracy_score, f1_score, classification_report

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Remove ignored index (special tokens like -100)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_preds = [[id2label[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "classification_report": classification_report(true_labels, true_preds)
    }

In [19]:
from transformers import Trainer, TrainingArguments


# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision
    gradient_accumulation_steps=4
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [21]:
import torch
torch.cuda.empty_cache()


In [22]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,No log


TrainOutput(global_step=212, training_loss=0.06629956443354769, metrics={'train_runtime': 552.5992, 'train_samples_per_second': 12.318, 'train_steps_per_second': 0.384, 'total_flos': 3150334307401728.0, 'train_loss': 0.06629956443354769, 'epoch': 0.9964747356051704})

# **Inference Pipeline with Aggregation Strategy**

In [23]:
from transformers import pipeline

# Load NER pipeline with trained model
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Example of using the pipeline
text = "John Doe's email is john.doe@example.com and his phone number is 123-456-7890."
ner_results = ner_pipeline(text)

print(ner_results)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LABEL_0', 'score': 0.9999436, 'word': "John Doe's email is john.doe@example.com and his phone number is 123-456-7890.", 'start': 0, 'end': 78}]
